::::
:::{thebe-button}
:::
::::

# Compare centers

Find bubble centers using two different approaches.

In [ ]:
from boilercv_docs.nbs import init

paths = init()

from devtools import pprint
from geopandas import GeoDataFrame, points_from_xy
from matplotlib.figure import Figure
from matplotlib.pyplot import subplot_mosaic
from myst_nb import glue
from pandas import DataFrame, IndexSlice, NamedAgg
from seaborn import scatterplot
from shapely import LinearRing

from boilercv.data import VIDEO
from boilercv.images import scale_bool
from boilercv_docs.nbs import HIDE, nowarn, set_display_options
from boilercv_pipeline.config import default
from boilercv_pipeline.experiments.e230920_subcool import GBC, bounded_ax
from boilercv_pipeline.models.notebooks import Notebooks
from boilercv_pipeline.sets import get_contours_df, get_dataset

default.__init__()  # noqa: PLC2801
p = default.notebooks

COMPARE_WITH_TRACKPY = True
"""Whether to get centers using the Trackpy approach."""

YMIN = 0
YMAX = 300
XMIN = 140
XMAX = 210

GUESS_DIAMETER = 41
"""Guess diameter for the Trackpy approach. (px)"""

FIGURES: list[Figure] = []
"""Notebook figures available for export."""

HIDE

## Bubble contours over time

Showing filled contours at a fixed frame interval illustrates bubbles rise and shrink as they depart the boiling surface. A small bubble that happens to be far from the surface at the beginning of the video also rises and continues to shrink, its origin presumed to have been at the boiling surface prior to the start of the video. In fact, bubbles generally grow and depart the surface at different times, and so a study of bubble histories must align these histories at their moment of departure from the boiling surface, and incomplete bubble histories filtered out.

In [ ]:
p = Notebooks.model_validate(p)
set_display_options(p.font_scale)
pprint(p)

In [ ]:
if COMPARE_WITH_TRACKPY:
    with nowarn(capture=True):
        from trackpy import batch, quiet

    quiet()

PATH_TIME = p.time.replace(":", "-")
"""Timestamp suitable for paths.

Also used in notebook parametrization.
"""

filled_contours = scale_bool(
    get_dataset(PATH_TIME, stage="filled", frame=p.frames).sel(
        ypx=slice(YMIN, YMAX), xpx=slice(XMIN, XMAX)
    )[VIDEO]
)
figure, axs = subplot_mosaic([["first_frame", "composite"]])
FIGURES.append(figure)
with bounded_ax(
    (composite := (filled_contours.max("frame").values)),
    axs["composite"],  # pyright: ignore[reportArgumentType]
) as composite_ax:  # pyright: ignore[reportArgumentType]
    composite_ax.imshow(~composite, alpha=0.4)
    xlim = composite_ax.get_xlim()
    ylim = composite_ax.get_ylim()
    composite_ax.set_xlabel("x (px)")
    composite_ax.set_ylabel("y (px)")
    axs["first_frame"].imshow(  # pyright: ignore[reportArgumentType]
        ~scale_bool(
            get_dataset(
                PATH_TIME, stage="large_sources", frame=slice(None, 0, None)
            ).sel(ypx=slice(YMIN, YMAX), xpx=slice(XMIN, XMAX))[VIDEO]
        ).squeeze()
    )
    axs["first_frame"].set_xlim(xlim)  # pyright: ignore[reportArgumentType]
    axs["first_frame"].set_ylim(ylim)  # pyright: ignore[reportArgumentType]
    axs["first_frame"].set_xlabel("x (px)")  # pyright: ignore[reportArgumentType]
    axs["first_frame"].set_ylabel("y (px)")  # pyright: ignore[reportArgumentType]

In [ ]:
glue("nb-2024-07-30-compare-centers_composite", figure, display=False)

```{glue:figure} nb-2024-07-30-compare-centers_composite
:name: fig.experiments.e230920_subcool.find_centers.composite
:alt: Bubbles shown to rise and shrink. One bubble is shown starting above the others.

Filled bubble contours at a constant frame interval indicating movement over time.
```

## Use Trackpy to find bubble centers

Trackpy is a pure Python implementation of the Crocker-Grier algorithm for identifying and tracking objects {cite}`allanTrackpy2018,crockerMethodsDigitalVideo1996`. The general approach involves locating Gaussian-like blobs in each frame of a video, then linking nearest neighbors. Bubbles

Compare experimental bubble histories to bubble history correlations. Bubble detection and linking performed by Trackpy, an implementation of the Crocker-Grier algorithm .


In [ ]:
if COMPARE_WITH_TRACKPY:
    trackpy_centers = (
        batch(
            frames=filled_contours.values, diameter=GUESS_DIAMETER, characterize=False
        )
        .drop(columns="mass")
        .assign(
            frame=lambda df: df.frame.replace(
                dict(enumerate(filled_contours.frame.values))
            )
        )
    )
else:
    trackpy_centers = DataFrame()

trackpy_centers.rename(columns={"frame": "Frame #"}).set_index("Frame #").rename_axis(
    columns="Centroid"
)

## Find centers from contour centroids

The prior approach throws out contour data, instead operating on filled contours. Instead, try using Shapely to find centers directly from contour data.


### Prepare to find objects

Prepare a dataframe with columns in a certain order, assign contour data to it, and demote the hiearchical indices to plain columns. Count the number of points in each contour and each frame, keeping only those which have enough points to describe a linear ring. Construct a GeoPandas geometry column and operate on it with Shapely to construct linear rings, returning only their centroids. Also report the number of points in the loci of each contour per frame.

:::{admonition} Implementation detail: `groupby` considerations  
:class: dropdown note  
`groupby` operations behave differently depending on the index, so resetting the index before grouping, and unpacking `GBC` to set sensible defaults for `groupby`'s keyword arguments, makes it behave less surprisingly. `GBC` enables `observed` and `sort`, and disables `as_index`, `dropna`, and `group_keys`.  
:::

In [ ]:
contours = (
    DataFrame(columns=["xpx", "ypx"])
    .assign(**get_contours_df(PATH_TIME).loc[IndexSlice[p.frames, :], :])
    .rename(axis="columns", mapper=dict(xpx="x", ypx="y"))
    .reset_index()
    .assign(
        count=lambda df: df.groupby(["frame", "contour"], **GBC).x.transform("count")
    )
    .query("count > 3")
    .assign(geometry=lambda df: points_from_xy(df.x, df.y))
    .groupby(["frame", "contour"], **GBC)
    .agg(
        count=NamedAgg(column="count", aggfunc="first"),
        centroid=NamedAgg(
            column="geometry", aggfunc=lambda df: LinearRing(df).centroid
        ),
    )
)

contours.set_index(["frame", "contour", "count"])

Split the centroid point objects into separate named columns that conform to the Trackpy convention. Report the centroids in each frame.


In [ ]:
centers = (
    GeoDataFrame(contours)
    .assign(x=lambda df: df.centroid.x, y=lambda df: df.centroid.y)
    .loc[:, ["y", "x", "frame"]]
    .sort_values(["frame", "y", "x"], ignore_index=True)
    .assign(x=lambda df: df.x - XMIN)
)

## Compare approaches

Compare Trackpy centers with contour centroids. Here the guess radius for Trackpy object finding and contour perimeter filtering are matched to produce the same number of objects from each algorithm. Trackpy features more intelligent filtering, but takes much longer. Trackpy's approach for finding local maxima in grayscale images is applied even to binarized images, exhaustively searching for high points in the binary image, adding to execution time. A warm color palette is used to plot Trackpy centers, and a cool color palette is used to plot contour centroids.

In [ ]:
if COMPARE_WITH_TRACKPY:
    scatterplot(
        ax=axs["composite"],  # pyright: ignore[reportArgumentType]
        data=trackpy_centers,
        x="x",
        y="y",
        s=p.size,
        alpha=0.5,
        color="red",
        legend=False,
    )
scatterplot(
    ax=axs["composite"],  # pyright: ignore[reportArgumentType]
    data=centers,
    x="x",
    y="y",
    s=p.size,
    alpha=0.5,
    color="blue",
    legend=False,
)

figure

In [ ]:
glue("nb-2024-07-30-compare-centers_compared", figure, display=False)

```{glue:figure} nb-2024-07-30-compare-centers_compared
:name: fig.experiments.e230920_subcool.find_centers.compared
:alt: Comparison of Trackpy and Shapely approaches to finding bubble centers.

Comparison of Trackpy and Shapely approaches to finding bubble centers.
```